## Evaluation Metric

In [1]:
import os
import cv2
import torch
import shutil
import random
import tempfile
import numpy as np
from PIL import Image
from skimage import io
from pathlib import Path
from monai.transforms import LoadImage
from pytorch_fid import fid_score
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim

## FID

In [5]:
'''
from glob import glob

def split_images(src_folder, dest_folder1):
    # Create destination folders if they don't exist
    os.makedirs(dest_folder1, exist_ok=True)
    
    # Get list of image files in the source folder
    image_files = glob(os.path.join(src_folder, '*.png'))
    
    # Split the image files list in half
    midpoint = 1800
    first_half = image_files[:midpoint]
    
    # Copy first half to the first destination folder
    for image_file in first_half:
        shutil.copy(image_file, dest_folder1)
        
    print(f'{len(first_half)} images copied to {dest_folder1}')

# Example usage:
src_folder = './generated_images/275_3.0'
dest_folder1 = './generated_images/275_3.0_1800'

split_images(src_folder, dest_folder1)
'''

1800 images copied to ./generated_images/275_3.0_1800


In [4]:
### Real
path_real = "./real_images/275_3.0_1800"
path_real_1 = "./real_images/275_3.0_900_1"
path_real_2 = "./real_images/275_3.0_900_2"
fid_value_real_real = fid_score.calculate_fid_given_paths([path_real_1, path_real_2], batch_size=50, device='cuda', dims=768)
print("FID real vs real:", fid_value_real_real)

### FID 7.0
path_generated = "./generated_images/275_7.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_7.0 real vs generated:", fid_value_real_generated)


### FID 5.0
path_generated = "./generated_images/275_5.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_5.0 real vs generated:", fid_value_real_generated)


### FID 3.0
path_generated = "./generated_images/275_3.0_1800"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_3.0 real vs generated:", fid_value_real_generated)


### FID 1.0
path_generated = "./generated_images/275_1.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_1.0 real vs generated:", fid_value_real_generated)


### FID 0.5
path_generated = "./generated_images/275_0.5"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_0.5 real vs generated:", fid_value_real_generated)


### FID 0.0
path_generated = "./generated_images/275_0.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=768)
print("FID_0.0 real vs generated:", fid_value_real_generated)

100%|███████████████████████████████████████████| 18/18 [00:01<00:00, 15.31it/s]


FID real vs real: 0.01383555504658951


100%|███████████████████████████████████████████| 36/36 [00:01<00:00, 18.32it/s]


FID_7.0 real vs generated: 0.0757485575104413


100%|███████████████████████████████████████████| 36/36 [00:01<00:00, 19.11it/s]


...

FID_1.0 real vs generated: 0.014486460377095611


100%|███████████████████████████████████████████| 36/36 [00:05<00:00,  6.78it/s]


FID_0.5 real vs generated: 0.014990837080128938


100%|███████████████████████████████████████████| 36/36 [00:05<00:00,  6.22it/s]


FID_0.0 real vs generated: 0.020649161585681697


## SSIM

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith('.png'):
            file_path = os.path.join(folder, filename)
            img = cv2.imread(file_path, 0)
            images.append(img)
    return images

def calculate_ssim(images):
    ssim_values = []
    image_count = len(images)
    
    # Generate all possible pairs without replacement
    indices = list(range(image_count))
    random.shuffle(indices)

    for i in range(image_count):
        for j in range(i + 1, image_count):
            img1 = images[indices[i]]
            img2 = images[indices[j]]

            # Compute SSIM
            ssim_value = ssim(img1, img2, data_range=255)
            ssim_values.append(ssim_value)

    return ssim_values

In [5]:
### Real
path_real = "./real_images/275_3.0_1800"
real_images = load_images_from_folder(path_real)
ssim_values_real_real = calculate_ssim(real_images)
print("Average SSIM real vs real:", np.mean(ssim_values_real_real))


### SSIM 7.0
path_generated = "./generated_images/275_7.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 7.0 generated vs generated:", np.mean(ssim_values_gen_gen))


### SSIM 5.0
path_generated = "./generated_images/275_5.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 5.0 generated vs generated:", np.mean(ssim_values_gen_gen))


### SSIM 3.0
path_generated = "./generated_images/275_3.0_1800"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 3.0 generated vs generated:", np.mean(ssim_values_gen_gen))


### SSIM 1.0
path_generated = "./generated_images/275_1.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 1.0 generated vs generated:", np.mean(ssim_values_gen_gen))


### SSIM 0.5
path_generated = "./generated_images/275_0.5"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 0.5 generated vs generated:", np.mean(ssim_values_gen_gen))


### SSIM 0.0
path_generated = "./generated_images/275_0.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 0.0 generated vs generated:", np.mean(ssim_values_gen_gen))

Average SSIM 7.0 real vs real: 0.547137986599241
Average SSIM 7.0 generated vs generated: 0.5142215728290703
Average SSIM 5.0 generated vs generated: 0.5310162602974586
Average SSIM 3.0 generated vs generated: 0.5468499829217688
Average SSIM 1.0 generated vs generated: 0.5508953664838239
Average SSIM 0.5 generated vs generated: 0.5615519239241891
Average SSIM 0.0 generated vs generated: 0.5590109095769186
